In [7]:
import sys
import os

# Giả sử bạn đang ở: /path/to/projects/notebooks
# Ta thêm thư mục Text2CAD vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../Text2CAD'))
sys.path.append(project_root)


In [8]:
import json
import pandas as pd
import numpy as np
import torch
from CadSeqProc.cad_sequence import CADSequence
from CadSeqProc.utility.utils import normalize_pc, chamfer_dist
from CadSeqProc.utility.macro import CAD_CLASS_INFO, N_BIT
from CadSeqProc.utility.utils import (create_path_with_time,ensure_dir)
import os
from tqdm import tqdm
import json
import math
from collections import defaultdict
from datasets import load_dataset


In [9]:
d = load_dataset("wanhin/test_multi_2k_l5")
d

DatasetDict({
    no_sampling_446600_448600: Dataset({
        features: ['id', 'input', 'reasoning', 'predicted_output', 'ground_truth_output'],
        num_rows: 2000
    })
})

In [10]:
x = 0
data = []
for r in d['no_sampling_446600_448600']:
    if r['predicted_output'] =='':
        x+= 1
    else:
        data.append(r)
x

72

In [11]:
data_en = []
data_vi = []
for r in data:
    if 'Suy' in r['reasoning']:
        data_vi.append(r)
    else:
        data_en.append(r)
print(len(data_en), '  -  ', len(data_vi))

950   -   978


In [35]:
data_en_ok = []
for sample in data_en[:200]:
    try:
        try:
            zzz = json.loads(sample['ground_truth_output'].replace("'",'"')[:-1])
            gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
        except:
            try:
                zzz = json.loads(sample['ground_truth_output'].replace("'",'"'))
                gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
            except:
                zzz = json.loads(sample['ground_truth_output'].replace("'",'"') + '}')
                gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
        zzz = json.loads(sample['predicted_output'].replace("'",'"'))
        pt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
    except:
        continue
    data_en_ok.append(sample)
len(data_en_ok)

181

In [36]:
data_vi_ok = []
for sample in data_vi[:200]:
    try:
        try:
            zzz = json.loads(sample['ground_truth_output'].replace("'",'"')[:-1])
            gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
        except:
            try:
                zzz = json.loads(sample['ground_truth_output'].replace("'",'"'))
                gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
            except:
                zzz = json.loads(sample['ground_truth_output'].replace("'",'"') + '}')
                gt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
        zzz = json.loads(sample['predicted_output'].replace("'",'"'))
        pt_cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
    except:
        continue
    data_vi_ok.append(sample)
len(data_vi_ok)

191

In [20]:
import re
import json

def extract_description_json_pairs(text):
    # Regex để tìm tất cả các cặp <description>...</description> và <json>...</json>
    description_pattern = re.compile(r"<description>\s*(.*?)\s*</description>", re.DOTALL)
    json_pattern = re.compile(r"<json>\s*(\{.*?\})\s*</json>", re.DOTALL)

    descriptions = description_pattern.findall(text)
    jsons = json_pattern.findall(text)

    # Trả về danh sách các cặp (description, json)
    pairs = []
    for desc, js in zip(descriptions, jsons):
        try:
            parsed_json = json.loads(js)
        except json.JSONDecodeError:
            parsed_json = js  # Giữ dạng text nếu không thể parse
        pairs.append({
            "description": desc.strip(),
            "json": parsed_json
        })

    return pairs

In [ ]:
path_save = './eval'

In [56]:
k  = 0
for sample in data_vi_ok:
    content = sample['input'].split('</instruction>')[-1]
    pairs = extract_description_json_pairs(content)
    final_desc = content.split('<description>')[-1].replace('</description>', '')
    final_json = json.loads(sample['predicted_output'].replace("'",'"'))
    pairs.append({
        "description": final_desc.strip(),
        "json": final_json
    })
    if len(pairs)>1:
        folder_path = f"./eval/vi/sample_{k}"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        text = ''
        i = 0
        for s in pairs:
            text += s['description'] +  '\n\n======================\n\n'
            zzz = s['json']
            try:
                cad = CADSequence.from_minimal_json(zzz).create_cad_model().sample_points(n_points=8192)
                cad.save_stp(f'{i}', folder_path)
            except:
                continue
            i += 1
        with open(folder_path + "/description.txt", "w", encoding="utf-8") as f:
            f.write(text)
        k+=1


In [4]:
content = """
 'input': '<objective>\nGenerate a JSON file describing the sketching and extrusion steps needed to construct a 3D CAD model based on the description provided. The output should include a reasoning section within the <think> tag and the corresponding JSON in the <json> tag. Do not provide any additional text outside of the tags.\n</objective>\n\n<instruction>\n    You will be given a natural language description of a CAD design task enclosed within the <description> tag. Additionally, you may also receive up to two previous turns of input/output context in <previous_turn_1> and <previous_turn_2>. These turns represent earlier steps in a multi-step CAD design process.\n\n    Each previous turn includes both the <think> and <json> tags, providing the model\'s reasoning and structured output from earlier steps.\n\n    - If the tags <previous_turn_1> or <previous_turn_2> are empty, it means there is no prior context for that slot.\n    - Use any available previous turns to inform geometric references, operation chaining, or any dependencies needed in the current modeling step.\n\n    Your task is to:\n\n    1. Analyze the current <description> and extract relevant geometric and extrusion information.\n    2. Reference earlier steps from <previous_turn_1> and <previous_turn_2> if available to maintain continuity and accuracy.\n    3. In the <think> tag, explain how you derived each field and value in the JSON, including reasoning for geometry, dependencies on earlier steps, and chosen operations.\n    4. In the <json> tag, return only the structured CAD instruction in JSON format corresponding to this design step.\n\n    The extrusion <operation> must be one of the following:\n\n    1. <NewBodyFeatureOperation>: Creates a new solid body.\n    2. <JoinFeatureOperation>: Fuses the shape with an existing body.\n    3. <CutFeatureOperation>: Subtracts the shape from an existing body.\n    4. <IntersectFeatureOperation>: Keeps only the overlapping volume between the new shape and existing body.\n</instruction>\n\n<previous_turn_1>\n<description>\nBây giờ tôi muốn thêm một phần khác, có vẻ là một cấu trúc dạng khung hoặc có các lỗ ở các cạnh, và nó sẽ kết nối hoặc gắn vào một số bộ phận hiện có.\n</description>\n\n<think>\nBước 1: Suy luận các thành phần sẽ có trong JSON dựa trên mô tả được cung cấp\nTừ yêu cầu "Bây giờ tôi muốn thêm một phần khác, có vẻ là một cấu trúc dạng khung hoặc có các lỗ ở các cạnh, và nó sẽ kết nối hoặc gắn vào một số bộ phận hiện có", tôi sẽ phân tích và giả định các yếu tố cần thiết, liệt kê các khóa JSON cụ thể cho `part_5`.\n\n**part_5**: Định nghĩa phần thứ năm.\n**coordinate_system**: Thành phần để định vị và hướng phần này trong không gian.\n  **Euler Angles**: Dựa trên input gốc (`[0.0,0.0,-90.0]`), tôi sẽ giữ nguyên giá trị này.\n  **Translation Vector**: Dựa trên input gốc (`[0.0,0.2962,0.0]`), tôi sẽ giữ nguyên giá trị này để đặt phần này vào một vị trí phù hợp.\n**sketch**: Thành phần định nghĩa bản phác thảo 2D.\n  **face_1**: Đại diện cho một biên dạng phức tạp.\n    **loop_1**: Biên dạng kín được tạo bởi các đoạn thẳng.\n      **line_1** đến **line_6**: Các đoạn thẳng này sẽ được định nghĩa với `Start Point` và `End Point` tương ứng với các giá trị trong input gốc.\n        `line_1: {"Start Point":[0.0,0.0],"End Point":[0.096,0.0]}`\n        `line_2: {"Start Point":[0.096,0.0],"End Point":[0.096,0.096]}`\n        `line_3: {"Start Point":[0.096,0.096],"End Point":[0.085,0.096]}`\n        `line_4: {"Start Point":[0.085,0.096],"End Point":[0.085,0.011]}`\n        `line_5: {"Start Point":[0.085,0.011],"End Point":[0.0,0.011]}`\n        `line_6: {"Start Point":[0.0,0.011],"End Point":[0.0,0.0]}`\n  **face_2**: Đại diện cho một biên dạng khác, cũng được tạo bởi các đoạn thẳng.\n    **loop_1**: Biên dạng kín.\n      **line_1** đến **line_6**: Các đoạn thẳng này sẽ được định nghĩa với `Start Point` và `End Point` tương ứng với các giá trị trong input gốc.\n        `line_1: {"Start Point":[0.3262,0.0],"End Point":[0.4222,0.0]}`\n        `line_2: {"Start Point":[0.4222,0.0],"End Point":[0.4222,0.011]}`\n        `line_3: {"Start Point":[0.4222,0.011],"End Point":[0.3372,0.011]}`\n        `line_4: {"Start Point":[0.3372,0.011],"End Point":[0.3372,0.096]}`\n        `line_5: {"Start Point":[0.3372,0.096],"End Point":[0.3262,0.096]}`\n        `line_6: {"Start Point":[0.3262,0.096],"End Point":[0.3262,0.0]}`\n    Mô tả "cấu trúc dạng khung hoặc có các lỗ ở các cạnh" có thể được thể hiện bằng các biên dạng phức tạp này, có thể là phần của một khung hoặc chi tiết có các cạnh không thẳng.\n**extrusion**: Thành phần định nghĩa quá trình đùn.\n  **extrude_depth_towards_normal**: Dựa trên input gốc (`0.2962`), tôi sẽ giữ nguyên giá trị này.\n  **extrude_depth_opposite_normal**: Dựa trên input gốc (`0.0`), ngụ ý việc đùn chỉ diễn ra theo một hướng.\n  **sketch_scale**: Dựa trên input gốc (`0.4222`), tôi sẽ giữ nguyên giá trị này.\n  **operation**: Dựa trên input gốc (`"JoinFeatureOperation"`), tôi sẽ giữ nguyên để chỉ định đây là thao tác ghép nối.\n\nBước 2: Kiểm tra logic, tính đúng đắn về số học, và thực hiện các sửa đổi (nếu cần thiết) từ Bước 1\nTôi kiểm tra tính hợp lý của các thông số đã suy luận từ Bước 1.\n\n**coordinate_system**:\n  **Euler Angles:[0.0,0.0,-90.0]**: Giữ nguyên từ input gốc. Kết luận: Hợp lý.\n  **Translation Vector:[0.0,0.2962,0.0]**: Giữ nguyên từ input gốc, xác định vị trí của phần này. Kết luận: Hợp lý.\n**sketch**:\n  **face_1/loop_1** và **face_2/loop_1**: Các đoạn thẳng đã được sao chép chính xác từ input gốc.\n    Kiểm tra tính liên tục cho face_1/loop_1:\n    - line_1: Start(0.0,0.0) -> End(0.096,0.0)\n    - line_2: Start(0.096,0.0) -> End(0.096,0.096)\n    - line_3: Start(0.096,0.096) -> End(0.085,0.096)\n    - line_4: Start(0.085,0.096) -> End(0.085,0.011)\n    - line_5: Start(0.085,0.011) -> End(0.0,0.011)\n    - line_6: Start(0.0,0.011) -> End(0.0,0.0) (kết nối lại với đầu line_1). Biên dạng kín.\n    Kiểm tra tính liên tục cho face_2/loop_1:\n    - line_1: Start(0.3262,0.0) -> End(0.4222,0.0)\n    - line_2: Start(0.4222,0.0) -> End(0.4222,0.011)\n    - line_3: Start(0.4222,0.011) -> End(0.3372,0.011)\n    - line_4: Start(0.3372,0.011) -> End(0.3372,0.096)\n    - line_5: Start(0.3372,0.096) -> End(0.3262,0.096)\n    - line_6: Start(0.3262,0.096) -> End(0.3262,0.0) (kết nối lại với đầu line_1). Biên dạng kín.\n    Hai biên dạng phức tạp được định nghĩa rõ ràng, phù hợp với mô tả "cấu trúc dạng khung hoặc có các lỗ ở các cạnh". Kết luận: Hợp lý.\n**extrusion**:\n  **extrude_depth_towards_normal:0.2962**: Chiều sâu đùn. Kết luận: Hợp lý.\n  **extrude_depth_opposite_normal:0.0**: Chỉ đùn một chiều. Kết luận: Hợp lý.\n  **sketch_scale:0.4222**: Tỷ lệ bản phác thảo. Kết luận: Hợp lý.\n  **operation:"JoinFeatureOperation"**: Ghép nối với các phần hiện có. Kết luận: Hợp lý.\nThực hiện các sửa đổi (nếu cần thiết): Không có thông số nào cần sửa đổi vì tất cả các giá trị đã được sao chép chính xác từ input gốc để tái tạo phần này.\n</think>\n\n<json>\n{"parts":{"part_1":{"coordinate_system":{"Euler Angles":[0.0,0.0,-90.0],"Translation Vector":[0.096,0.2962,0.0]},"sketch":{"face_1":{"loop_1":{"line_1":{"Start Point":[0.0,0.0],"End Point":[0.1371,0.0]},"line_2":{"Start Point":[0.1371,0.0],"End Point":[0.1371,0.1816]},"line_3":{"Start Point":[0.1371,0.1816],"End Point":[0.1928,0.1816]},"line_4":{"Start Point":[0.1928,0.1816],"End Point":[0.1928,0.0]},"line_5":{"Start Point":[0.1928,0.0],"End Point":[0.2303,0.0]},"line_6":{"Start Point":[0.2303,0.0],"End Point":[0.2303,0.3736]},"line_7":{"Start Point":[0.2303,0.3736],"End Point":[0.0,0.3736]},"line_8":{"Start Point":[0.0,0.3736],"End Point":[0.0,0.0]}}}},"extrusion":{"extrude_depth_towards_normal":0.2578,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.3736,"operation":"NewBodyFeatureOperation"}},"part_2":{"coordinate_system":{"Euler Angles":[180.0,0.0,-90.0],"Translation Vector":[0.2617,0.2962,0.3736]},"sketch":{"face_1":{"loop_1":{"line_1":{"Start Point":[0.0,0.0],"End Point":[0.1657,0.0]},"line_2":{"Start Point":[0.1657,0.0],"End Point":[0.1657,0.0123]},"line_3":{"Start Point":[0.1657,0.0123],"End Point":[0.0,0.0123]},"line_4":{"Start Point":[0.0,0.0123],"End Point":[0.0,0.0]}}}},"extrusion":{"extrude_depth_towards_normal":0.0,"extrude_depth_opposite_normal":0.1657,"sketch_scale":0.1657,"operation":"NewBodyFeatureOperation"}},"part_3":{"coordinate_system":{"Euler Angles":[0.0,0.0,0.0],"Translation Vector":[0.1033,0.1374,0.3859]},"sketch":{"face_1":{"loop_1":{"circle_1":{"Center":[0.0758,0.0758],"Radius":0.0758}}}},"extrusion":{"extrude_depth_towards_normal":0.3202,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.1515,"operation":"JoinFeatureOperation"}},"part_4":{"coordinate_system":{"Euler Angles":[0.0,0.0,0.0],"Translation Vector":[0.1279,0.1614,0.7061]},"sketch":{"face_1":{"loop_1":{"circle_1":{"Center":[0.0514,0.0514],"Radius":0.0514}}}},"extrusion":{"extrude_depth_towards_normal":0.0439,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.1029,"operation":"JoinFeatureOperation"}},"part_5":{"coordinate_system":{"Euler Angles":[0.0,0.0,-90.0],"Translation Vector":[0.0,0.2962,0.0]},"sketch":{"face_1":{"loop_1":{"line_1":{"Start Point":[0.0,0.0],"End Point":[0.096,0.0]},"line_2":{"Start Point":[0.096,0.0],"End Point":[0.096,0.096]},"line_3":{"Start Point":[0.096,0.096],"End Point":[0.085,0.096]},"line_4":{"Start Point":[0.085,0.096],"End Point":[0.085,0.011]},"line_5":{"Start Point":[0.085,0.011],"End Point":[0.0,0.011]},"line_6":{"Start Point":[0.0,0.011],"End Point":[0.0,0.0]}}},"face_2":{"loop_1":{"line_1":{"Start Point":[0.3262,0.0],"End Point":[0.4222,0.0]},"line_2":{"Start Point":[0.4222,0.0],"End Point":[0.4222,0.011]},"line_3":{"Start Point":[0.4222,0.011],"End Point":[0.3372,0.011]},"line_4":{"Start Point":[0.3372,0.011],"End Point":[0.3372,0.096]},"line_5":{"Start Point":[0.3372,0.096],"End Point":[0.3262,0.096]},"line_6":{"Start Point":[0.3262,0.096],"End Point":[0.3262,0.0]}}}},"extrusion":{"extrude_depth_towards_normal":0.2962,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.4222,"operation":"JoinFeatureOperation"}}}}\n</json>\n</previous_turn_1>\n\n<previous_turn_2>\n<description>\nBây giờ tôi muốn thêm một cái gì đó nhỏ nữa, có vẻ như là một đinh tán hoặc một trụ nhỏ nhô ra, ở vị trí khác trên mô hình.\n</description>\n\n<think>\nBước 1: Suy luận các thành phần sẽ có trong JSON dựa trên mô tả được cung cấp\nTừ yêu cầu "Bây giờ tôi muốn thêm một cái gì đó nhỏ nữa, có vẻ như là một đinh tán hoặc một trụ nhỏ nhô ra, ở vị trí khác trên mô hình", tôi sẽ phân tích và giả định các yếu tố cần thiết, liệt kê các khóa JSON cụ thể cho `part_4`.\n\n**part_4**: Định nghĩa phần thứ tư.\n**coordinate_system**: Thành phần để định vị và hướng phần này trong không gian.\n  **Euler Angles**: Dựa trên input gốc (`[0.0,0.0,0.0]`), tôi sẽ giữ nguyên giá trị này.\n  **Translation Vector**: Dựa trên input gốc (`[0.1279,0.1614,0.7061]`), tôi sẽ giữ nguyên giá trị này để đặt trụ nhỏ ở vị trí khác.\n**sketch**: Thành phần định nghĩa bản phác thảo 2D cho trụ nhỏ.\n  **face_1**: Đại diện cho mặt phẳng chứa bản phác thảo.\n    **loop_1**: Đại diện cho biên dạng của trụ nhỏ.\n      **circle_1**: Là hình dạng của trụ nhỏ.\n        **Center**: Dựa trên input gốc (`[0.0514,0.0514]`), tôi sẽ giữ nguyên giá trị này để xác định tâm của trụ nhỏ.\n        **Radius**: Dựa trên input gốc (`0.0514`), tôi sẽ giữ nguyên giá trị này để xác định bán kính của trụ nhỏ.\n**extrusion**: Thành phần định nghĩa quá trình đùn để tạo trụ nhỏ.\n  **extrude_depth_towards_normal**: Dựa trên input gốc (`0.0439`), tôi sẽ giữ nguyên giá trị này để xác định chiều cao của trụ nhỏ.\n  **extrude_depth_opposite_normal**: Dựa trên input gốc (`0.0`), ngụ ý việc đùn chỉ diễn ra theo một hướng.\n  **sketch_scale**: Dựa trên input gốc (`0.1029`), tôi sẽ giữ nguyên giá trị này.\n  **operation**: Dựa trên input gốc (`"JoinFeatureOperation"`), tôi sẽ giữ nguyên để chỉ định đây là thao tác ghép nối với các phần hiện có.\n\nBước 2: Kiểm tra logic, tính đúng đắn về số học, và thực hiện các sửa đổi (nếu cần thiết) từ Bước 1\nTôi kiểm tra tính hợp lý của các thông số đã suy luận từ Bước 1.\n\n**coordinate_system**:\n  **Euler Angles:[0.0,0.0,0.0]**: Giữ nguyên từ input gốc, không có xoay. Kết luận: Hợp lý.\n  **Translation Vector:[0.1279,0.1614,0.7061]**: Giữ nguyên từ input gốc, xác định vị trí của trụ nhỏ. Kết luận: Hợp lý.\n**sketch**:\n  **face_1/loop_1/circle_1**:\n    **Center:[0.0514,0.0514]**: Xác định tâm trụ nhỏ. Kết luận: Hợp lý.\n    **Radius:0.0514**: Xác định bán kính trụ nhỏ. Kết luận: Hợp lý.\n    Hình dạng tròn đơn giản được định nghĩa đúng.\n**extrusion**:\n  **extrude_depth_towards_normal:0.0439**: Chiều cao của trụ nhỏ. Kết luận: Hợp lý.\n  **extrude_depth_opposite_normal:0.0**: Chỉ đùn một chiều. Kết luận: Hợp lý.\n  **sketch_scale:0.1029**: Tỷ lệ bản phác thảo hợp lý. Kết luận: Hợp lý.\n  **operation:"JoinFeatureOperation"**: Ghép nối với các phần hiện có. Kết luận: Hợp lý.\nThực hiện các sửa đổi (nếu cần thiết): Không có thông số nào cần sửa đổi vì tất cả các giá trị đã được sao chép chính xác từ input gốc để tái tạo trụ nhỏ.\n</think>\n\n<json>\n{"parts":{"part_1":{"coordinate_system":{"Euler Angles":[0.0,0.0,-90.0],"Translation Vector":[0.096,0.2962,0.0]},"sketch":{"face_1":{"loop_1":{"line_1":{"Start Point":[0.0,0.0],"End Point":[0.1371,0.0]},"line_2":{"Start Point":[0.1371,0.0],"End Point":[0.1371,0.1816]},"line_3":{"Start Point":[0.1371,0.1816],"End Point":[0.1928,0.1816]},"line_4":{"Start Point":[0.1928,0.1816],"End Point":[0.1928,0.0]},"line_5":{"Start Point":[0.1928,0.0],"End Point":[0.2303,0.0]},"line_6":{"Start Point":[0.2303,0.0],"End Point":[0.2303,0.3736]},"line_7":{"Start Point":[0.2303,0.3736],"End Point":[0.0,0.3736]},"line_8":{"Start Point":[0.0,0.3736],"End Point":[0.0,0.0]}}}},"extrusion":{"extrude_depth_towards_normal":0.2578,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.3736,"operation":"NewBodyFeatureOperation"}},"part_2":{"coordinate_system":{"Euler Angles":[180.0,0.0,-90.0],"Translation Vector":[0.2617,0.2962,0.3736]},"sketch":{"face_1":{"loop_1":{"line_1":{"Start Point":[0.0,0.0],"End Point":[0.1657,0.0]},"line_2":{"Start Point":[0.1657,0.0],"End Point":[0.1657,0.0123]},"line_3":{"Start Point":[0.1657,0.0123],"End Point":[0.0,0.0123]},"line_4":{"Start Point":[0.0,0.0123],"End Point":[0.0,0.0]}}}},"extrusion":{"extrude_depth_towards_normal":0.0,"extrude_depth_opposite_normal":0.1657,"sketch_scale":0.1657,"operation":"NewBodyFeatureOperation"}},"part_3":{"coordinate_system":{"Euler Angles":[0.0,0.0,0.0],"Translation Vector":[0.1033,0.1374,0.3859]},"sketch":{"face_1":{"loop_1":{"circle_1":{"Center":[0.0758,0.0758],"Radius":0.0758}}}},"extrusion":{"extrude_depth_towards_normal":0.3202,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.1515,"operation":"JoinFeatureOperation"}},"part_4":{"coordinate_system":{"Euler Angles":[0.0,0.0,0.0],"Translation Vector":[0.1279,0.1614,0.7061]},"sketch":{"face_1":{"loop_1":{"circle_1":{"Center":[0.0514,0.0514],"Radius":0.0514}}}},"extrusion":{"extrude_depth_towards_normal":0.0439,"extrude_depth_opposite_normal":0.0,"sketch_scale":0.1029,"operation":"JoinFeatureOperation"}}}}\n</json>\n</previous_turn_2>\n\n<description>\nCuối cùng, tôi muốn thêm bốn lỗ tròn nhỏ, được tạo ra bằng cách cắt xuyên qua một phần của mô hình, ở các vị trí khác nhau và cách đều nhau.\n</description>',

"""
content = content.split('</instruction>')[-1]




In [6]:
content.split('<description>')[-1]

"\nCuối cùng, tôi muốn thêm bốn lỗ tròn nhỏ, được tạo ra bằng cách cắt xuyên qua một phần của mô hình, ở các vị trí khác nhau và cách đều nhau.\n</description>',\n\n"

In [5]:




pairs = extract_description_json_pairs(content)

# Xuất thử kết quả
for i, pair in enumerate(pairs, 1):
    print(f"--- Pair {i} ---")
    print("Description:")
    print(pair["description"])
    print("JSON:")
    print(json.dumps(pair["json"], indent=2) if isinstance(pair["json"], dict) else pair["json"])


--- Pair 1 ---
Description:
Bây giờ tôi muốn thêm một phần khác, có vẻ là một cấu trúc dạng khung hoặc có các lỗ ở các cạnh, và nó sẽ kết nối hoặc gắn vào một số bộ phận hiện có.
JSON:
{
  "parts": {
    "part_1": {
      "coordinate_system": {
        "Euler Angles": [
          0.0,
          0.0,
          -90.0
        ],
        "Translation Vector": [
          0.096,
          0.2962,
          0.0
        ]
      },
      "sketch": {
        "face_1": {
          "loop_1": {
            "line_1": {
              "Start Point": [
                0.0,
                0.0
              ],
              "End Point": [
                0.1371,
                0.0
              ]
            },
            "line_2": {
              "Start Point": [
                0.1371,
                0.0
              ],
              "End Point": [
                0.1371,
                0.1816
              ]
            },
            "line_3": {
              "Start Point": [
                0